In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [ ]:
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

## Transformers and Estimators

### Transformers - Tokenizer

In [ ]:
from pyspark.ml.feature import Tokenizer

In [ ]:
sentenceDataFrame = sqlc.createDataFrame([(0, "Hi I heard about Spark"),
                                          (1, "I wish Java could use case classes"),
                                          (2, "Logistic,regression,models,are,neat")]) \
                    .toDF("label", "sentence")

In [ ]:
tokenizer = Tokenizer().setInputCol("sentence").setOutputCol("words")
tokenized = tokenizer.transform(sentenceDataFrame)

In [ ]:
tokenized.toPandas()

### Transformers - Vector Assembler

In [ ]:
from pyspark.sql.functions import rand, randn
from pyspark.ml.feature import VectorAssembler

dfRandom = sqlc.range(0, 10).select("id") \
            .withColumn("uniform", rand(10)) \
            .withColumn("normal1", randn(10)) \
            .withColumn("normal2", randn(11))

In [ ]:
assembler = VectorAssembler(inputCols = ["uniform","normal1","normal2"], outputCol = "features")

dfVec = assembler.transform(dfRandom)

In [ ]:
dfVec.select("id","features").toPandas()

### Estimator - Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import Vectors

In [ ]:
training = sqlc.createDataFrame([(1.0, Vectors.dense(0.0, 1.1, 0.1)),
                                       (0.0, Vectors.dense(2.0, 1.0, -1.0)),
                                       (0.0, Vectors.dense(2.0, 1.3, 1.0)),
                                       (1.0, Vectors.dense(0.0, 1.2, -0.5))]) \
            .toDF("label", "features")

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.setMaxIter(10).setRegParam(0.01)

In [ ]:
model1 = lr.fit(training, {'maxIter': 10, 'regParam': 0.01})
model1.coefficients

In [ ]:
model1.transform(training).toPandas()

In [ ]:
sc.stop()